# 합성 테스트 데이터셋 생성

**왜 합성 테스트 데이터(Synthetic Test Dataset) 인가?**

RAG(검색 증강 생성) 증강 파이프라인의 성능을 평가하는 것은 매우 중요합니다. 

그러나 문서에서 수백 개의 QA(질문-문맥-응답) 샘플을 수동으로 생성하는 것은 시간과 노동력이 많이 소요될 수 있습니다. 또한 사람이 만든 질문은 철저한 평가에 필요한 복잡성 수준에 도달하기 어려워 궁극적으로 평가의 품질에 영향을 미칠 수 있습니다. 

합성 데이터 생성을 사용하면 데이터 집계 프로세스에서 **개발자의 시간을 90%** 까지 줄일 수 있습니다.

- RAGAS: https://docs.ragas.io/en/latest/concepts/testset_generation.html

아래의 주석을 해제한 후 실행하여 패키지를 설치 후 진행해주세요

**참고**

(2024.10.08 현재) ragas 패키지는 0.2.16 버전의 langchain 패키지와 호환됩니다.

따라서 버전을 0.2.16 버전으로 설치해야 합니다. (현재, 0.3.x 버전으로 우리가 사용하는 버전이 더 상위 버전입니다)

아래의 주석을 해제 후 진행해 주세요!

In [1]:
import langchain
import ragas

print(f"LangChain Version: {langchain.__version__}")
print(f"Ragas Version: {ragas.__version__}")

LangChain Version: 0.2.16
Ragas Version: 0.1.21


In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

_실습을 위해 다운로드 받은 파일을 `data` 폴더로 복사해 주시기 바랍니다_


## 문서 전처리

문서를 로드 합니다.

In [3]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("/workspace/rag-baseline/data/card/0_iM Social Worker카드.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
# docs = docs[3:-1]

# 문서의 페이지수
len(docs)

6

각 문서 객체에는 `metadata` 를 통해 액세스할 수 있는 문서에 대한 추가 정보를 저장하는 데 사용할 수 있는 메타데이터 사전이 포함되어 있습니다. 

메타데이터 사전에는 `filename` 이라는 키가 포함되어 있는지 확인하세요. 

이 키는 Test datasets 생성 프로세스에서 활용될 것이므로. 메타데이터의 `filename` 속성은 동일한 문서에 속한 청크를 식별하는 데 사용됩니다. 

In [4]:
docs[0].metadata

{'source': '/workspace/rag-baseline/data/card/0_iM Social Worker카드.pdf',
 'file_path': '/workspace/rag-baseline/data/card/0_iM Social Worker카드.pdf',
 'page': 0,
 'total_pages': 6,
 'CreationDate': "D:20240827155544+10'00'",
 'Creator': 'Adobe Illustrator 24.1 (Windows)',
 'ModDate': "D:20240828104209+09'00'",
 'Producer': 'Adobe PDF library 15.00',
 'Title': '(240826) iM Social Worker카드-영업점'}

In [5]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

## 데이터셋 생성

In [6]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기
generator_llm = ChatOpenAI(model="gpt-4o-mini")
# 데이터셋 비평기
critic_llm = ChatOpenAI(model="gpt-4o-mini")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

DocumentStore를 초기화합니다. 사용자 정의 LLM과 임베딩을 사용합니다.

In [7]:
# 텍스트 분할기를 설정합니다.
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# LangChain의 ChatOpenAI 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요 구문 추출기를 초기화합니다. 위에서 정의한 LLM을 사용합니다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InMemoryDocumentStore를 초기화합니다.
# 이는 문서를 메모리에 저장하고 관리하는 저장소입니다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor,
)

TestSet 을 생성합니다.

In [8]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore,
)

**질문의 유형별 분포**

- simple: 간단한 질문
- reasoning: 추론이 필요한 질문
- multi_context: 여러 맥락을 고려해야 하는 질문
- conditional: 조건부 질문

In [9]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}

- documents: 문서 데이터
- test_size: 생성할 질문의 수
- distributions: 질문 유형별 분포
- with_debugging_logs: 디버깅 로그 출력 여부

In [10]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs,
    test_size=10,
    distributions=distributions,
    with_debugging_logs=True,
    raise_exceptions=False,
)

embedding nodes:   0%|          | 0/26 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 1, 'structure': 1, 'relevance': 1, 'score': 1.0}
[ragas.testset.evolutions.INFO] retrying evolution: 0 times
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['연회비', '카드발급', '신용카드', '신용카드 이용', '신용카드 혜택']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 1, 'structure': 1, 'relevance': 1, 'score': 1.0}
[ragas.testset.evolutions.INFO] retrying evolution: 0 times
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['신용카드', 'ARS', '상용형', '결제 시스템', '고객센터']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 2, 'score': 1.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['상품별 설명서', 

In [11]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the purpose of using a shared credit c...,"[2. 이용대금 상환방법, 금리(수수료) 및 변동 여부 5. 계약 해지 시 불이익\...",The answer to given question is not present in...,simple,[{'source': '/workspace/rag-baseline/data/card...,True
1,What is the purpose of the application of stan...,[10. 이용대금 결제일 전 이용대금 상환이 필요한 경우 12. 위법계약해지권\n□...,The answer to given question is not present in...,simple,[{'source': '/workspace/rag-baseline/data/card...,True
2,What are the applications of synthetic fibers?,"[2. 이용대금 상환방법, 금리(수수료) 및 변동 여부 5. 계약 해지 시 불이익\...",The answer to given question is not present in...,simple,[{'source': '/workspace/rag-baseline/data/card...,True
3,What is the significance of the term 'annual f...,[준법감시인 심의필 제23-1524호(2023.12.19~2025.12.18)\n고...,The term 'annual fee' (연회비) in the context of ...,simple,[{'source': '/workspace/rag-baseline/data/card...,True
4,What properties of synthetic fibers affect the...,[1. 신용카드의 특성\n□ 신용카드는 회원의 신용을 담보로 신용카드 가맹점에서 상...,The properties of synthetic fibers that affect...,reasoning,[{'source': '/workspace/rag-baseline/data/card...,True
5,How does the usage card system relate to resou...,[1. 신용카드의 특성\n□ 신용카드는 회원의 신용을 담보로 신용카드 가맹점에서 상...,The answer to given question is not present in...,reasoning,[{'source': '/workspace/rag-baseline/data/card...,True
6,How does performance management aid resource u...,"[(예시) 연회비 2만원, 반환금액 산정 제외비용 1만원, 연회비 부과대상일수 금리...",The answer to given question is not present in...,multi_context,[{'source': '/workspace/rag-baseline/data/card...,True
7,What drives the usage card system in transacti...,"[2. 이용대금 상환방법, 금리(수수료) 및 변동 여부 5. 계약 해지 시 불이익\...",The context does not provide specific informat...,multi_context,[{'source': '/workspace/rag-baseline/data/card...,True
8,How does '\uc2e0\uc6a9\uc815\ubcf4' affect ass...,[볼빙) 이월 잔액 + 당월 일부결제 (50만원 + 30만원) 관리 > BC카드 카...,The context does not provide information on ho...,conditional,[{'source': '/workspace/rag-baseline/data/card...,True
9,How does a usage card affect transactions for ...,[준법감시인 심의필 제23-1524호(2023.12.19~2025.12.18)\n고...,The usage card affects transactions for a spec...,conditional,[{'source': '/workspace/rag-baseline/data/card...,True


DataFrame 에 저장된 데이터셋을 csv 파일로 저장합니다

In [12]:
# DataFrame의 상위 5개 행 출력
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the purpose of using a shared credit c...,"[2. 이용대금 상환방법, 금리(수수료) 및 변동 여부 5. 계약 해지 시 불이익\...",The answer to given question is not present in...,simple,[{'source': '/workspace/rag-baseline/data/card...,True
1,What is the purpose of the application of stan...,[10. 이용대금 결제일 전 이용대금 상환이 필요한 경우 12. 위법계약해지권\n□...,The answer to given question is not present in...,simple,[{'source': '/workspace/rag-baseline/data/card...,True
2,What are the applications of synthetic fibers?,"[2. 이용대금 상환방법, 금리(수수료) 및 변동 여부 5. 계약 해지 시 불이익\...",The answer to given question is not present in...,simple,[{'source': '/workspace/rag-baseline/data/card...,True
3,What is the significance of the term 'annual f...,[준법감시인 심의필 제23-1524호(2023.12.19~2025.12.18)\n고...,The term 'annual fee' (연회비) in the context of ...,simple,[{'source': '/workspace/rag-baseline/data/card...,True
4,What properties of synthetic fibers affect the...,[1. 신용카드의 특성\n□ 신용카드는 회원의 신용을 담보로 신용카드 가맹점에서 상...,The properties of synthetic fibers that affect...,reasoning,[{'source': '/workspace/rag-baseline/data/card...,True


In [14]:
# DataFrame을 CSV 파일로 저장
import os

path = "./data/card"
test_df.to_csv(f"{path}/ragas_synthetic_data.csv", index=False, encoding="utf-8")